In [1]:
import json
import csv
import tweepy
import re

In [2]:
import pandas as pd
import numpy as np

In [12]:
#!pip install tweepy

## Get the content from Twitter API

In [3]:
"""
INPUTS:
    consumer_key, consumer_secret, access_token, access_token_secret: codes 
    telling twitter that we are authorized to access this data
    hashtag_phrase: the combination of hashtags to search for
OUTPUTS:
    none, simply save the tweet info to a spreadsheet
"""
def search_for_hashtags(consumer_key, consumer_secret, access_token, access_token_secret, hashtag_phrase):
    
    #create authentication for accessing Twitter
    auth = tweepy.OAuthHandler(consumer_key, consumer_secret)
    auth.set_access_token(access_token, access_token_secret)

    #initialize Tweepy API
    api = tweepy.API(auth)
    
    #get the name of the spreadsheet we will write to
    fname = '_'.join(re.findall(r"#(\w+)", hashtag_phrase))
    
    ls=[]
    for tweet in tweepy.Cursor(api.search, q=hashtag_phrase+' -filter:retweets', \
                                   lang="en", tweet_mode='extended').items(60):
         ls.append([tweet.created_at, tweet.full_text.replace('\n',' ').encode('utf-8').decode(), tweet.user.screen_name.encode('utf-8').decode(), [e['text'] for e in tweet._json['entities']['hashtags']], tweet.user.followers_count])
    return ls

In [4]:
import sys
consumer_key = "jFWAxHs4E02tO5jeBpleLCKry"
consumer_secret = "rlYaMztapJggVVZ4IpzaS1rykdI1Bqk6MeEgeS6Gz9it8oKP1B"
access_token = "4459940535-ny2IKAaSVjxDmtw5EMT9Lnj6SQgIrQT2sDOqY5T"
access_token_secret = "Rsk2wvNu7g4CsSv8r89KQxRubSFm2YAe7uy8nZOf8LpGQ"
    
hashtag_phrase = "#370 AND #kashmir"

if __name__ == '__main__':
    new_ls=search_for_hashtags(consumer_key, consumer_secret, access_token, access_token_secret, hashtag_phrase)

In [5]:
new_ls[:2]

[[datetime.datetime(2019, 10, 7, 21, 36, 42),
  'Kashmir: In #BurhanWani’s Home, No One Speaks Of Article 370  An afternoon with Burhan Wani’s family, post #Article370  #LetKashmirSpeak  @UN @antonioguterres @FedericaMog @MBachelet @UNHumanRights @HRW @MushaalMullick @peaceforchange @SoStarMusic #Kashmir https://t.co/rMTrkZ7roY',
  'HumanRightsPath',
  ['BurhanWani', 'Article370', 'LetKashmirSpeak', 'Kashmir'],
  18485],
 [datetime.datetime(2019, 10, 7, 19, 9, 42),
  '@Bagi56093020 @DrBurns10 @HouseForeign With the unilateral revocation of article 370 by India that accession has now ended therefore, #Kashmir is no more part of India, it’s now occupied by India. It’s you who needs history lessons',
  'annrosh_',
  ['Kashmir'],
  72]]

In [232]:
df=pd.DataFrame(new_ls,columns=['Timestamp','Tweet','User','Hashtag','FollowersCount'])

In [233]:
pd.set_option("display.max_colwidth",-1)

In [234]:
df.head()

,Timestamp,Tweet,User,Hashtag,FollowersCount
0,2019-10-07 21:36:42,"Kashmir: In #BurhanWani’s Home, No One Speaks Of Article 370 An afternoon with Burhan Wani’s family, post #Article370 #LetKashmirSpeak @UN @antonioguterres @FedericaMog @MBachelet @UNHumanRights @HRW @MushaalMullick @peaceforchange @SoStarMusic #Kashmir https://t.co/rMTrkZ7roY",HumanRightsPath,"[BurhanWani, Article370, LetKashmirSpeak, Kashmir]",18485
1,2019-10-07 19:09:42,"@Bagi56093020 @DrBurns10 @HouseForeign With the unilateral revocation of article 370 by India that accession has now ended therefore, #Kashmir is no more part of India, it’s now occupied by India. It’s you who needs history lessons",annrosh_,[Kashmir],72
2,2019-10-07 19:01:07,"LGBT activists storm anti-India meeting in London which Kavita Krishnan was attending. Gay, lesbian activists support scrapping of homophobic Article 370. This is an aspect of #Kashmir that jihad apologists will never speak about. https://t.co/Nsq7N5ywnU",abhijitmajumder,[Kashmir],530333
3,2019-10-07 18:44:10,"@HouseForeign Abrogation of article 370 IS to provide equal rights to all Kashmiris (including women, LGBT, Minorities) Do your homework before spewing venom with half baked knowledge!!! #Kashmir",nakshatra1812,[Kashmir],169
4,2019-10-07 16:58:38,"@Abhishek_0305 @KashmirOsint Because of #Kashmir, Jammu and Ladakh never saw any development. 370 was a violation of their democratic rights, yet people keep focusing only on #Kashmir as if they are some special species different other humans. Shame",AamTweet,"[Kashmir, Kashmir]",23


In [16]:
positive_words=pd.read_table("positive-words.txt")

In [21]:
negative_words=pd.read_csv("negative-words.txt")

## Clean the Data 

In [235]:
import re
l=[]
for i in df.Tweet:
    text=re.sub("(@[\w]*)|(#[\w]*)",'',i)
#     text=re.sub(r'@[\w]*','',i)
#     text=re.sub(r'#[\w]*','',text)
#     text=re.sub(r'\w+:\/\/\S+*','',text)
    text=" ".join(text.split())
    l.append(text.strip())

In [236]:
l[:2]

['Kashmir: In ’s Home, No One Speaks Of Article 370 An afternoon with Burhan Wani’s family, post https://t.co/rMTrkZ7roY',
 'With the unilateral revocation of article 370 by India that accession has now ended therefore, is no more part of India, it’s now occupied by India. It’s you who needs history lessons']

In [237]:
m=[]
for i in l:
    text=re.sub(r'\w+:\/\/\S+','',i)
    text=" ".join(text.split())
    m.append(text)

In [238]:
m[30]

'"I visited the suburb of Anchaar on the outskirts of Srinagar from 25 September through 27 September. The people there are desperate and resilient," writes as she brings us images from .'

In [239]:
df.Tweet=m

In [240]:
p=[]
for i in df.Tweet:
#     text=re.sub(r'!','',i)
#     text=re.sub(r'\(','',text)
#     text=re.sub(r'\)','',text)
    text=re.sub(r'[!\(\),\?;:]','',i)
    p.append(text)

In [241]:
p[3]

'Abrogation of article 370 IS to provide equal rights to all Kashmiris including women LGBT Minorities Do your homework before spewing venom with half baked knowledge'

In [242]:
df.Tweet=p

## Assign Sentiment using TextBlob

In [79]:
from textblob import TextBlob

In [243]:
sentiment=[]
for i in df.Tweet:
    i = TextBlob(i)
    if i.sentiment.polarity >0:
        sentiment.append(2)
        #print("Positive")
    elif i.sentiment.polarity ==0:
        sentiment.append(1)
        #print("Neutral")
    else:
        sentiment.append(0)
        #print("Negative")

In [244]:
df['sentiment']=sentiment

In [245]:
df.head()

,Timestamp,Tweet,User,Hashtag,FollowersCount,sentiment
0,2019-10-07 21:36:42,Kashmir In ’s Home No One Speaks Of Article 370 An afternoon with Burhan Wani’s family post,HumanRightsPath,"[BurhanWani, Article370, LetKashmirSpeak, Kashmir]",18485,1
1,2019-10-07 19:09:42,With the unilateral revocation of article 370 by India that accession has now ended therefore is no more part of India it’s now occupied by India. It’s you who needs history lessons,annrosh_,[Kashmir],72,0
2,2019-10-07 19:01:07,LGBT activists storm anti-India meeting in London which Kavita Krishnan was attending. Gay lesbian activists support scrapping of homophobic Article 370. This is an aspect of that jihad apologists will never speak about.,abhijitmajumder,[Kashmir],530333,2
3,2019-10-07 18:44:10,Abrogation of article 370 IS to provide equal rights to all Kashmiris including women LGBT Minorities Do your homework before spewing venom with half baked knowledge,nakshatra1812,[Kashmir],169,0
4,2019-10-07 16:58:38,Because of Jammu and Ladakh never saw any development. 370 was a violation of their democratic rights yet people keep focusing only on as if they are some special species different other humans. Shame,AamTweet,"[Kashmir, Kashmir]",23,2


In [246]:
Tweet=list(df['Tweet'].str.lower())

In [247]:
Tweet[:3]

['kashmir in ’s home no one speaks of article 370 an afternoon with burhan wani’s family post',
 'with the unilateral revocation of article 370 by india that accession has now ended therefore is no more part of india it’s now occupied by india. it’s you who needs history lessons',
 'lgbt activists storm anti-india meeting in london which kavita krishnan was attending. gay lesbian activists support scrapping of homophobic article 370. this is an aspect of that jihad apologists will never speak about.']

## Remove the stopwords

In [248]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.decomposition import TruncatedSVD
from nltk.corpus import stopwords
SW = set(stopwords.words('english'))
import spacy

In [249]:
nlp=spacy.load("en_core_web_md")

In [250]:
Tweet[:3]

['kashmir in ’s home no one speaks of article 370 an afternoon with burhan wani’s family post',
 'with the unilateral revocation of article 370 by india that accession has now ended therefore is no more part of india it’s now occupied by india. it’s you who needs history lessons',
 'lgbt activists storm anti-india meeting in london which kavita krishnan was attending. gay lesbian activists support scrapping of homophobic article 370. this is an aspect of that jihad apologists will never speak about.']

In [251]:
corpus=[]
for i in nlp.pipe(Tweet):
    doc_new=[]
    #doc1=nlp(i)
    for token in i:
        #print(i.pos_)
        if token.is_alpha and not(token.is_stop) and not(token.lemma_ in SW) and not(token.lemma_=="-PRON-"):
            doc_new.append(token.lemma_)
    corpus.append(" ".join(doc_new).strip())

In [252]:
corpus[2]

'lgbt activist storm anti india meeting london kavita krishnan attend gay lesbian activist support scrapping homophobic article aspect jihad apologist speak'

## Train-Test Split

In [257]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(corpus, df.sentiment, stratify = df.sentiment, test_size = 0.3, random_state = 0)

In [258]:
from sklearn.feature_extraction.text import TfidfVectorizer
tfidfVec = TfidfVectorizer().fit(X_train)

In [262]:
X_train = tfidfVec.transform(X_train)
X_test = tfidfVec.transform(X_test)

In [263]:
y_train.value_counts()

2    20
0    12
1    10
Name: sentiment, dtype: int64

In [264]:
y_test.value_counts()

2    9
0    5
1    4
Name: sentiment, dtype: int64

## SMOTE

### Multinomial Naive Bayes

In [268]:
import imblearn as imb
from imblearn.over_sampling import SMOTE
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import roc_auc_score, classification_report

SMT = SMOTE()

X_t, y_t = SMT.fit_sample(X_train, y_train)

MNB = MultinomialNB().fit(X_t, y_t)
y_pred_SMOTE = MNB.predict(X_test)

print("Statistics for classification are as follows: \n\n{}\n".format(classification_report(y_test, y_pred_SMOTE)))
#print("ROC AUC Score {}".format(roc_auc_score(y_test, y_pred_SMOTE)))

Statistics for classification are as follows: 

              precision    recall  f1-score   support

           0       0.33      0.20      0.25         5
           1       0.17      0.25      0.20         4
           2       0.56      0.56      0.56         9

   micro avg       0.39      0.39      0.39        18
   macro avg       0.35      0.34      0.34        18
weighted avg       0.41      0.39      0.39        18




### Logistic Regression

In [219]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import roc_curve

clfr = LogisticRegression(C = 100)
clfr.fit(X_t, y_t)

y_pred_proba = clfr.predict_proba(X_test)
y_prob = []
for i in y_pred_proba:
    y_prob.append(i[1])

fpr, tpr, thr = roc_curve(y_test, y_prob, pos_label = 1)

C:\Users\sneha chikhalekar\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


## Normal Multinomial naive bayes

In [266]:
from sklearn.naive_bayes import MultinomialNB

from sklearn.metrics import roc_auc_score, classification_report
MNB = MultinomialNB().fit(X_train, y_train)
y_pred = MNB.predict(X_test)

print("Statistics for classification are as follows: \n\n{}\n".format(classification_report(y_test, y_pred)))
#print("ROC AUC Score {}".format(roc_auc_score(y_test, y_pred)))

C:\Users\sneha chikhalekar\Anaconda3\lib\site-packages\sklearn\metrics\classification.py:1143: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


Statistics for classification are as follows: 

              precision    recall  f1-score   support

           0       0.00      0.00      0.00         5
           1       0.00      0.00      0.00         4
           2       0.50      1.00      0.67         9

   micro avg       0.50      0.50      0.50        18
   macro avg       0.17      0.33      0.22        18
weighted avg       0.25      0.50      0.33        18




In [316]:
from sklearn.model_selection import train_test_split
X_tr, X_ts, y_tr, y_ts = train_test_split(corpus, df.sentiment, stratify = df.sentiment, test_size = 0.3, random_state = 0)
indices = X_ts.index
pd.DataFrame({'Tweet':X_ts, 'Prediction':y_pred_SMOTE, 'Test':y_ts})

,Tweet,Prediction,Test
56,pakistan fuel kashmir separatism article people happy javadekar gt sure people talk accord info siege food amp med leave,2,2
21,abrogation article unacceptable say nagaland leader,1,1
12,new normal glimpse image anchaar,2,2
40,today mark month revoke read theprint write day ago situation currently show security force high alert kid avoid school free movement access internet,2,2
5,finally bar tourist lift,2,1
15,people stop discuss article demand govt restore scrap special status suffering cross limit gloom amp disappointment people want fight issue resolve,1,2
39,want peace stability development matter art thing mention matter region prosper,1,2
17,bring resistance face world,0,1
44,lawmaker amp congress delegation visit hand condition local result article india lawmaker visit pok assess situation india article flipboard,0,2
2,lgbt activist storm anti india meeting london kavita krishnan attend gay lesbian activist support scrapping homophobic article aspect jihad apologist speak,2,2


In [315]:
X_ts[:3]

['pakistan fuel kashmir separatism article people happy javadekar gt sure people talk accord info siege food amp med leave',
 'abrogation article unacceptable say nagaland leader',
 'new normal glimpse image anchaar']

## LSA

In [269]:
tfidfVec=TfidfVectorizer().fit(corpus)
tfidfMat=tfidfVec.transform(corpus)

In [270]:
# Number of topics to extract
# Let's select the top 10 topics
ntopics = 10
svd_model = TruncatedSVD(n_components = ntopics, algorithm = "randomized")
svd_model.fit(tfidfMat)
svd_model

TruncatedSVD(algorithm='randomized', n_components=10, n_iter=5,
       random_state=None, tol=0.0)

In [271]:
svd_model.components_.shape

(10, 521)

In [272]:
# Get the names of the features in order to figure out the important
# terms in the respective topics in subsequent SVD analysis
terms = tfidfVec.get_feature_names()

In [273]:
# Get the weights of features(terms) in individual topics

for i, comp in enumerate(svd_model.components_):
    # Get a list of tuples with first entry as a term and second entry 
    # as it's weight in the ith topic
    term_component_tuple = zip(terms, comp)
    important_terms = sorted(term_component_tuple, key = lambda x:x[1], reverse = True)[:5]
    print("\nTopic {}: Top 5 important terms\n".format(str(i)))
    print("{0:<15}".format("Term"), "{0:<15}".format("Weight of Term"))
    for i in important_terms:
        print("{0:<15}".format(i[0]), "{:.3f}".format(i[1])) 


Topic 0: Top 5 important terms

Term            Weight of Term 
normal          0.417
people          0.256
amp             0.241
happy           0.241
say             0.227

Topic 1: Top 5 important terms

Term            Weight of Term 
kashmir         0.338
petition        0.315
wake            0.315
court           0.314
supreme         0.314

Topic 2: Top 5 important terms

Term            Weight of Term 
anchaar         0.456
image           0.431
glimpse         0.413
new             0.406
normal          0.190

Topic 3: Top 5 important terms

Term            Weight of Term 
govt            0.235
pm              0.234
back            0.231
correct         0.231
harish          0.231

Topic 4: Top 5 important terms

Term            Weight of Term 
back            0.208
correct         0.208
harish          0.208
mistake         0.208
salve           0.208

Topic 5: Top 5 important terms

Term            Weight of Term 
india           0.394
pm              0.115
back            